# Inheritance of Python classes

---

This notebook is about the inheritance logic and method which can be applied to Python classes.

---

## 1. Extension and modification of existing classes

When using classes there is always the wish to extend some previous classes or modify the behaviour of classes and objects. At this point you need to decide if you want to have these new features for all new created objects or if you want to have objects with the older and newer features.

If you have the source code available you can simply add the modifications by adding a new method or apply some new arguments for existing methods. If you don't have the source code then this may be problematic.

Assume that you want to have a homogeneous list of floats:

In [3]:
l = [1., 2., 3., 4.]

print(l)

l.append(10)          # append an integer, not homogenous anymore

l.append(float(10))   # use explicitly a conversion

print(l)

[1.0, 2.0, 3.0, 4.0]
[1.0, 2.0, 3.0, 4.0, 10, 10.0]


If you forget to convert values, you can append any types of values and the list will become inhomogeneous:

In [4]:
# solution of a homogeneous float list

class FloatList(list):
    def push(self, value):
        self.append(float(value))
        
        
l = FloatList([1., 2., 3., 4.])

l.push(10)

print(l)

[1.0, 2.0, 3.0, 4.0, 10.0]


Now you have a new list type which is called a stack with `pushes` a value at the end and if you use `.pop()` you can also remove the last element.

---

## 2. Simple Inheritance of an existing class

The simplest way to add an extension to an existing class is, to create a new class which is inherited from an existing class and add a new method:

Syntax of a new **inherited**  class:

```
class NewClass(OldClass):
   def newmethod(self,<arguments>):
       ...
```

In the new `method` you can adress all internal variables (if known) and methods with the `self` argument.

----

## 3. Overwriting exiting methods of inherited classes

The previous example is nice if you want to implement a stack like object in Python, but more generally when using `FloatList` you want to use `.append` with the conversion to float:

Naively one would write such a code:

```
class FloatList(list):
    def append(self, value):
        self.append(float(value))
```

Obviously inside the `append` method `self.append` will call the same function again and again. This recursion will never ends. The basic idea is to call the original `append` method from the `list` class.


To call the original class method, you need to call `append` from `list` and give the `instance` `self` as an additional argument:

In [37]:
# solution with class defintion calls

class FloatList(list):
    def append(self, value):
        list.append(self, float(value))
        
        
l = FloatList([1., 2., 3., 4.])

l.append(10)   # add a new value

print(l)

[1.0, 2.0, 3.0, 4.0, 10.0]


For this case, Python provides a better solution `super()` for which you don't need to know the name of the `original` class name, if you call  the original `append` method:

In [7]:
# solution with class defintion calls

class FloatList(list):
    def append(self, value):
        super().append(float(value))   # using super() for calling list.append
        
        
l = FloatList([1., 2., 3., 4.])

l.append(10)   # add a new value

print(l)

[1.0, 2.0, 3.0, 4.0, 10.0]


The last thing to make a full `FloatList` behavior is the initialization method, which should check and maybe converts values into floats:

In [12]:
# solution with class defintion calls

class FloatList(list):
    def __init__(self, values):        # value should be iterable
        float_values = [float(item) for item in values]

        # call the original method with the float values
        super().__init__(float_values)  
        
        
    def append(self, value):
        super().append(float(value))   # using super() for calling list.append
        
        
        
l = FloatList([1, 2, 3, 4])            # initialize the array with a list of integers

l.append(10)   # add a new value

print(l)

[1.0, 2.0, 3.0, 4.0, 10.0]


Now the new class should be complete.

---

## 4. Another example of inheritance

Often as an example for object orientated programming and inheritance `Person` as a class is mentioned. 

<center>
<img src="figs/person.png" style="width:75%">
</center>

Implementing the structure from `class` to `class`:

In [1]:
class Person(object):
    def __init__(self, lastname, firstname):
        self._lastname = lastname
        self._firstname = firstname
        
    def get_name(self):
        return f'{self._firstname} {self._lastname}'
    
    

a = Person('Cordes', 'Oliver')

print(a.get_name())
        

Oliver Cordes


The method `get_name` should be modified for the inheritated classes, so that all information of the class will be provided:

In [2]:
class Student(Person):
    def __init__(self, lastname, firstname, uniid):
        super().__init__(lastname, firstname)
        self._uniid = uniid
            
    def get_name(self):
        return f'{super().get_name()} ({self._uniid})'
    
    
b = Student('Cordes', 'Oliver', 'ocordes')

print(b.get_name())

Oliver Cordes (ocordes)


and finally:

In [21]:
class Professor(Person):
    def __init__(self, lastname, firstname, department):
        super().__init__(lastname, firstname)
        self._department = department
            
    def get_name(self):
        return f'Prof. {super().get_name()}'
    
    def get_department(self):
        return self._department
    
    
c = Professor('Cordes', 'Oliver', 'AIfA')

print(c.get_name())

Prof. Oliver Cordes


One application of such a definition is, that you can now manage a list of Persons which have different `implementations`, so Professors, Students and normal Persons:

In [22]:
persons = []

# create a list of mixed persons
persons.append(Professor('Cordes', 'Oliver', 'AIfA'))
persons.append(Professor('Erben', 'Thomas', 'AIfA'))
persons.append(Student('Go', 'Hu', 'hugo'))
persons.append(Student('Ta', 'Ber', 'berta'))
persons.append(Person('Cho', 'Ma' ))
persons.append(Person('My', 'Dum'))
               
# because they share all the same method get_name
for p in persons:
    print(p.get_name())
    
print()

# select the students
students = [p for p in persons if isinstance(p, Student)]

print('Students:')
for p in students:
    print(p.get_name())

print()

    
# select the professors only
profs = [p for p in persons if isinstance(p, Professor)]

print('Professors:')
for p in profs:
    print(p.get_name())
    print(p.get_department())  # only available for professors
print()

    
# select all persons ...
all = [p for p in persons if isinstance(p, Person)]
print(len(all))     # of course all are persons!

Prof. Oliver Cordes
Prof. Thomas Erben
Hu Go (hugo)
Ber Ta (berta)
Ma Cho
Dum My

Students:
Hu Go (hugo)
Ber Ta (berta)

Professors:
Prof. Oliver Cordes
AIfA
Prof. Thomas Erben
AIfA

6


**Note**: In this small program we call a method, which is available in the base class and all inherited classes. In C/C++ you need to implement these methods as `virtual`, because usually you need to define exactly during compile time, which class will be called. The `virtual` will help the compiler to call the correct method for each `instance`. Python as a `realtime`  interpreter know directly from the `instance` which method will be called!

Interestingly for some tests, it is necessary to check the type of an instance. If you need e.g. only the methods or attributes of the original base class, you can check for the base class only or you check for the inheritated classes, if you need special methods!

In [15]:
c = Professor('Cordes', 'Oliver', 'AIfA')

print(type(c))

print(isinstance(c, Professor))   # yes
print(isinstance(c, Person))      # also yes!
print(isinstance(c, Student))     # obviously no

# but
print(type(c) == Person)

<class '__main__.Professor'>
True
True
False
False


---

## 5. Cross inheritance

Similar to other object orientated programming languages, Python allows also the inheritance of multiple base classes. For our `Persons` example, we can define a special class, which handles all university courses and the credit points:

In [25]:
class Courses(object):
    def __init__(self, courses):
        self._courses = { c:0 for c in courses}
        
    def set_credit_points(self, name, cp):
        if name in self._courses:
            self._courses[name] = cp
            
    def __str__(self):
        s = ''
        for c in self._courses:
            s += f'  {c}: {self._courses[c]}\n'
        return s
            
c = Courses(['physics725', 'astro852'])

c.set_credit_points('physics725', 6)

print(c)

  physics725: 6
  astro852: 0



So now we want to define a new `Student` class which contains not only the `uniid` but also the complex `Courses` system:

In [41]:
class Student(Person, Courses):
    def __init__(self, lastname, firstname, uniid, courses):
        Person.__init__(self, lastname, firstname)
        Courses.__init__(self, courses)
        self._uniid = uniid
            
    def get_name(self):
        return f'{super().get_name()} ({self._uniid})'
    
    def __str__(self):
        return f'{self.get_name()}\n{Courses.__str__(self)}'
    
    
d = Student('Go', 'Hu', 'hugo', ['physics725', 'astro852'])
d.set_credit_points('physics725', 6)

print(d)

Hu Go (hugo)
  physics725: 6
  astro852: 0



**Important:** The previously defined `super()`-Function is not working in this case. As far as I can tell, `super()` will call all methods for all base classes, but always with the same argument list. The definition of the methods may be different! In such a case the base class needs to be called directly!

Problematic of the Cross inheritance is `compability`. Assume two different class definitions:

In [42]:
class ClassA(object):
    def __init__(self, A):
        self._property = A
        
        
class ClassB(object):
    def __init__(self, B):
        self._property = B
        
        
class ClassMix(ClassA, ClassB):
    def __init__(self, A, B):
        ClassA.__init__(self, A)
        ClassB.__init__(self, B)
                        
    def __str__(self):
        return self._property
                        
                        
c = ClassMix('A', 'B')
                        
print(c)
    

B


`ClassA` and `ClassB` are working perfectly if used independent of each other. When combining both classes there is the risk, that e.g. two properties can influence each other. 

**Note**: Before you combine two classes, check that all properties of the individual classes are not overlapping. In general it is the same problem when new classes are defined as inherited classes. 

One solution would be not to inherit many base classes but include special instances as properties:

In [43]:
class ClassA(object):
    def __init__(self, A):
        self._property = A
        
        
class ClassB(object):
    def __init__(self, B):
        self._property = B
        
        
class ClassMix(ClassA, ClassB):
    def __init__(self, A, B):
        self._A = ClassA(A)
        self._B = ClassB(B)

                        
    def __str__(self):
        return f'{self._A._property} {self._B._property}'
                        
                        
c = ClassMix('A', 'B')
                        
print(c)

A B


---

## 6. Summary for inheritance

 * you can overwrite any method directly
 * use `super()` if you want to call a method from the original class
 * Python don't know about `virtual` methods, since it uses an interpreter always the correct method will be called
 * you can inherit from multiple base classes, `super()` is not working properly, use base class definitions
 * keep an eye on previously defined properties and methods that the inheritance not influence the classes (e.g. double used properties)